In [57]:
import os

In [60]:
import ultralytics
ultralytics.__version__

'8.3.21'

In [61]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import os


os.environ["YOLO_VERBOSE"] = "false"


model=YOLO('yolov5n.pt')

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [62]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [63]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [64]:
#llegir el video
cap=cv2.VideoCapture('output2.mp4')

In [65]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]
downwards = 0
upwards = 0
line_y = 400  # Coordenada y de la línia única
frames = 0
current_fps = 0


In [67]:
import time
import numpy as np

start_time = time.time()
pts_ROI = np.array([[359, 480],[186, 480] ,[8, 840], [446, 841]], np.int32)
pts_ROI = pts_ROI // 2
pts_ROI = pts_ROI.reshape((-1, 1, 2))

frame_difference = np.array([])

pts = np.array([[359, 340],[186, 371] ,[8, 953], [446, 951]], np.int32)
# divide all values by 2
pts = pts // 2
pts = pts.reshape((-1, 1, 2))

last_frame = None

while True:
    skip = False
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    # RESOLUCIO IMATGE ORIGINAL: 960X540
    # BAIXAR RESOLUCIO
    frame = cv2.resize(frame, (270, 480))
    original_frame = frame.copy()

    mask = np.zeros_like(frame, dtype=np.uint8)
    cv2.fillPoly(mask, [pts], (255, 255, 255))
    roi_frame = cv2.bitwise_and(original_frame, mask)
    
    if last_frame is not None:
        frame_difference = cv2.absdiff(last_frame, frame)*2


        frame_difference = cv2.bitwise_and(frame_difference, mask)

        print(frame_difference.mean())

        # cv2.imshow("frame_difference", frame_difference)

    # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
    if count % 2 != 0 or frame_difference.mean() < 1:
            cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # red line
            if frame_difference.mean() > 1:
                cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0, 255), 2) 
            cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            

    # Convertir el frame a grisos per anar més ràpid: 
    # No es pot utilitzar perquè YOLO treballa en RGB
    # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    else:
        # YOLO Prediction
        results = model.predict(roi_frame)

        # aconseguint les dades de la bounding box i confiança
        a = results[0].boxes.data
        a = a.detach().cpu().numpy()
        px = pd.DataFrame(a).astype("float")

        # only work with car class. only count cars. Quedarnos bounding box dels cars
        car_bboxes = []
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])
            c = class_list[d]
            if 'car' in c:
                car_bboxes.append([x1, y1, x2, y2])

        # provide les coordinades al tracker
        bbox_id = tracker.update(car_bboxes)

        # draw bbox
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int((x3 + x4) // 2)
            cy = int((y3 + y4) // 2)

            # Actualitzar comptadors de cotxes que baixen o pugen
            offset = 7  # Margen d'error per la detecció de passos per la línia

            # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
            if cy < line_y - offset:  # Cotxe a la part superior de la línia
                down[id] = cy  # Guardar la y del cotxe quan està a la part superior
            if id in down:
                if cy > line_y + offset:  # Cotxe passa a la part inferior
                    # Dibuixar un punt vermell al cotxe que baixa
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

            # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
            if cy > line_y + offset:  # Cotxe a la part inferior de la línia
                up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
            if id in up:
                if cy < line_y - offset:  # Cotxe passa a la part superior
                    # Dibuixar un punt vermell al cotxe que puja
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                    # Mostrar l'ID del cotxe al frame
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

            # Dibuixar la línia vermella al frame
        cv2.line(frame, (0, line_y), (800, line_y), (0, 0, 255), 3)  # línia vermella (única)
        cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Show counters on the frame
    downwards = len(set(counter_down))
    upwards = len(set(counter_up))



    # FPS calculation
    #count the fps by counting the number of frames per second 
    frames +=1
    if frames == 20:
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            current_fps = 20/elapsed_time
        start_time = time.time()
        frames = 0
        
    cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        
    cv2.imshow("frames", frame)

    
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
    last_frame = original_frame

cap.release()
cv2.destroyAllWindows()


1.9547427983539094



0: 640x384 2 cars, 77.3ms
Speed: 2.0ms preprocess, 77.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
1.7687808641975309
1.8136574074074074



C:\Users\polri\AppData\Local\Temp\ipykernel_30708\873572710.py:49: RuntimeWarning: Mean of empty slice.
  if frame_difference.mean() > 1:
c:\Users\polri\Documents\00_Uni\4rt Curs - 2024_25\1r semestre\Mètodes Avançats de Processament de Senyal, Imatge i Vídeo\01_Reptes\Repte2_Cotxes\PSIV_Repte2\venv\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0: 640x384 2 cars, 75.1ms
Speed: 2.0ms preprocess, 75.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
1.636388888888889
1.7682458847736626

0: 640x384 2 cars, 1 truck, 76.5ms
Speed: 2.0ms preprocess, 76.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
2.094022633744856
1.7749639917695472

0: 640x384 2 cars, 77.0ms
Speed: 1.0ms preprocess, 77.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
2.4187860082304526
1.8711574074074073

0: 640x384 2 cars, 80.8ms
Speed: 2.0ms preprocess, 80.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
1.7978909465020576
1.9708847736625514

0: 640x384 2 cars, 67.6ms
Speed: 1.9ms preprocess, 67.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
1.7607458847736626
1.7774794238683127

0: 640x384 2 cars, 77.0ms
Speed: 1.9ms preprocess, 77.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
1.7915072016460905
1.7858179012345679

0: 640x384 2 cars, 89

KeyboardInterrupt: 

In [15]:
cap.release()
cv2.destroyAllWindows()